In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df.shape

(50000, 2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [7]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [8]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

the task is to classify a given review as positive or negative by understanding the sentiment of the text written in the review

In [9]:
# steps 

# 1) text preprocessing (removing html tags , stopwords etc)
# 2) vectorize the preprocessed text (vectorization the process of converting the text to numbers) eg -> BagOfWords , TF-IDF , WordToVec
# 3) apply naive bayes 

# 1) Text Preprocessing

1) take 10000 rows from all te avialable rows
2) remove html tags
3) remove special characters
4) remove stopwords
5) convert to lowercase
6) Stemming

In [10]:
df = df.sample(10000)

In [11]:
df.shape

(10000, 2)

In [12]:
df['sentiment'].replace({'positive':1,'negative':0},inplace=True)

/tmp/ipykernel_13/4240984564.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['sentiment'].replace({'positive':1,'negative':0},inplace=True)
/tmp/ipykernel_13/4240984564.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'].replace({'positive':1,'negative':0},inplace=True)


In [13]:
df.head()

,review,sentiment
43560,Whenever a Columbo story deviates from the fam...,0
19788,Sadly a great opportunity to utilise a superb ...,0
42384,"Pretentious claptrap, updating Herman Melville...",0
23445,"I'm a big fan of Naruto, even though I haven't...",1
12887,In trying to keep up with the hipness of youth...,0


In [14]:
# demo of removing html tags
import re 
cleaned = re.compile('<*.?>')
re.sub(cleaned,'',df.iloc[2].review)

"Pretentious claptrap, updating Herman Melville (!), about a young man's vaguely incestuous relationship with his aristocratic mother getting transferred to his long-lost sister who has been raised by gypsies. Or something like that \x96 not that anyone really cares to unravel its multi-layered plot decked out with pornographic sex scenes, pseudo-symbolic imagery (the siblings swimming in a river of blood) and other bizarre touches (a gypsy child repeatedly insults passers-by in the street until she is anonymously beaten to death, the deafening music of a rock group utilized in the demolition of old buildings). Considering the source material and the presence of Catherine Deneuve (who at least gets to bathe in the nude), I was expecting a lot more from this one; apparently, there's an even longer TV version of POLA X out there\x85"

In [15]:
def remove_tags(text):
    clean = re.compile('<*>?>')
    return re.sub(clean,'',text)

df['review']=df['review'].apply(remove_tags)
    

In [16]:
# convert to lower case
def tolower(text):
    return text.lower()

df['review']=df['review'].apply(tolower)

In [17]:
df.head()

,review,sentiment
43560,whenever a columbo story deviates from the fam...,0
19788,sadly a great opportunity to utilise a superb ...,0
42384,"pretentious claptrap, updating herman melville...",0
23445,"i'm a big fan of naruto, even though i haven't...",1
12887,in trying to keep up with the hipness of youth...,0


In [18]:
# remove the special characters from the text 

def remove_special(text):
    removed = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return removed

df['review']=df['review'].apply(remove_special)

In [19]:
df['review'].head()

# the text is now in lower case doesnot have special characters and html tags

43560    whenever a columbo story deviates from the fam...
19788    sadly a great opportunity to utilise a superb ...
42384    pretentious claptrap updating herman melville ...
23445    im a big fan of naruto even though i havent wa...
12887    in trying to keep up with the hipness of youth...
Name: review, dtype: object

In [20]:
# removing the stopwords 
# stopwords are the words which do not have any meaning in the sentence as we count the unique words. eg -> is , and , the 

import nltk
from nltk.corpus import stopwords
stopwords.words('english')


# below are the set of words which will be removed as stopwords

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [21]:
# removing stopwords 

stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    removed_stopwords =[]
    for i in text.split():
        if i not in stop_words:
            removed_stopwords.append(i)
    return removed_stopwords

df['review']= df['review'].apply(remove_stopwords)

In [22]:
df.head()

,review,sentiment
43560,"[whenever, columbo, story, deviates, familiar,...",0
19788,"[sadly, great, opportunity, utilise, superb, c...",0
42384,"[pretentious, claptrap, updating, herman, melv...",0
23445,"[im, big, fan, naruto, even, though, havent, w...",1
12887,"[trying, keep, hipness, youthful, audiences, 7...",0


In [23]:
# Stemming 
# stemming is the process of converting the actual word to the root word 

from nltk.stem import PorterStemmer
porter = PorterStemmer()


def stemming(text):
    stemmed = []
    for i in text:
        stemmed.append(porter.stem(i))
    return stemmed 
    
df['review'] = df['review'].apply(stemming)

In [24]:
df.head()

,review,sentiment
43560,"[whenev, columbo, stori, deviat, familiar, plo...",0
19788,"[sadli, great, opportun, utilis, superb, cast,...",0
42384,"[pretenti, claptrap, updat, herman, melvil, yo...",0
23445,"[im, big, fan, naruto, even, though, havent, w...",1
12887,"[tri, keep, hip, youth, audienc, 70, approach,...",0


In [25]:
# join back the words from the list together to form a sentence again

def join_back(list_input):
    return " ".join(list_input)

df['review']=df['review'].apply(join_back)

In [26]:
df.head()

,review,sentiment
43560,whenev columbo stori deviat familiar plot colo...,0
19788,sadli great opportun utilis superb cast bring ...,0
42384,pretenti claptrap updat herman melvil young ma...,0
23445,im big fan naruto even though havent watch eve...,1
12887,tri keep hip youth audienc 70 approach oacdycs...,0


# 2) Vectorization

in  bag of words , the entire available data is joined by the algorithm in to  a single big string  , and in the complete string , first the algorithm finds the number of unique words ,  and then finds the frequency of the unique words . 
now for every individual text the algorithm counts the occurrence of every word . 
and this counting is a numerical representation of every individual text , and hence the given textual data is converted into numerical data 

In [27]:
df.head()

,review,sentiment
43560,whenev columbo stori deviat familiar plot colo...,0
19788,sadli great opportun utilis superb cast bring ...,0
42384,pretenti claptrap updat herman melvil young ma...,0
23445,im big fan naruto even though havent watch eve...,1
12887,tri keep hip youth audienc 70 approach oacdycs...,0


In [28]:
# converting the first column (review) after text preprocessing into the 2D numpy array
X = df.iloc[:,0:1].values

In [29]:
X.shape

(10000, 1)

In [30]:
# get a particular number of unique words 

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)  # get 2500 unique features

X=cv.fit_transform(df['review']).toarray()  

In [31]:
X.shape  # the newshape has 2500 as number of unique rows

(10000, 2500)

now every textual review is in the from of numbers

In [32]:
X[0].mean()  # mean of the numbers of the frst review

0.016

In [33]:
y  = df.iloc[:,-1].values

In [34]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2)

In [35]:
print(X_train.shape,X_test.shape)

(8000, 2500) (2000, 2500)


# Applying Naive Bayes

In [36]:
from sklearn.naive_bayes import GaussianNB , MultinomialNB , BernoulliNB

In [37]:
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()


In [38]:
clf1.fit(X_train ,y_train)
clf2.fit(X_train , y_train)
clf3.fit(X_train , y_train)

BernoulliNB()

In [39]:
y_pred1 = clf1.predict(X_test)
y_pred2 = clf2.predict(X_test)
y_pred3 = clf3.predict(X_test)

In [40]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred1))
print(accuracy_score(y_test,y_pred2))
print(accuracy_score(y_test,y_pred3))

0.731
0.834
0.843
